In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [ ]:
from keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomContrast, RandomBrightness, RandomCrop, Rescaling

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../data/train',
    image_size=(48, 48),
    batch_size=128,
    label_mode='categorical',
    validation_split=0.1,
    subset='training',
    seed=0,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../data/train',
    image_size=(48, 48),
    batch_size=128,
    label_mode='categorical',
    validation_split=0.1,
    subset='validation',
    seed=0,
    
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    '../data/test',
    image_size=(48, 48),
    batch_size=128,
    label_mode='categorical',
    seed=0,
)

# Rescale pixel values
normalization_layer = Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Data augmentation
data_augmentation = tf.keras.Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.1),
    RandomZoom(0.1),
    RandomContrast(0.1),
    RandomBrightness(0.1),
])

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y))

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, InputLayer, ReLU
from keras.models import Sequential

model = Sequential([
    InputLayer(input_shape=(48, 48, 3)),
    Conv2D(32, (3, 3), padding='same'),
    BatchNormalization(),
    ReLU(),
    Dropout(0.2),
    Conv2D(32, (3, 3), padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    ReLU(),
    Dropout(0.2),
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    ReLU(),
    Dropout(0.2),
    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    ReLU(),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    
    Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [4]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=15, restore_best_weights=True, monitor='val_loss')
learning_rate_reduction = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=7, monitor='val_loss')
hist = model.fit(train_ds, validation_data=val_ds, epochs=50)

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(hist.history['accuracy'], label='accuracy')
plt.plot(hist.history['val_accuracy'], label='val_accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(hist.history['loss'], label='loss')
plt.plot(hist.history['val_loss'], label='val_loss')
plt.legend()

plt.show()

In [ ]:
# Test accuracy
print(f"Model evaluation on test data: {model.evaluate(test_ds)[1]:.3f}")

In [38]:
# Save the model
# model.save('models/model.h5')